In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
import utils
from pandas import Timedelta
import time
from utils import logger, sql_to_df, run_sql, session, conn
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

05:43:10 INFO:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: macOS-10.15.7-x86_64-i386-64bit
05:43:10 INFO:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
05:43:10 INFO:Setting use_openssl_only mode to False
05:43:12 INFO:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: macOS-10.15.7-x86_64-i386-64bit
05:43:12 INFO:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


connected to account OMWYKHW-US_WEST_2 wh DEMO db SHARE_COPY schema KNOT_MONITORING with role DEV


05:43:13 INFO:Snowpark Session information: 
"version" : 1.3.0,
"python.version" : 3.8.16,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 9098488785367046,
"os.name" : Darwin



time: 19.1 ms (started: 2023-06-06 17:43:13 -04:00)


In [3]:
staging_tables = [
    'stg_rate_sheet_daily',
    'stg_warehouse_metering_history',
    'stg_access_history',
    'stg_grants_to_roles',
    'stg_query_history',
    'stg_remaining_balance_daily',
    'stg_sessions',
    'stg_storage_usage',
    'stg_table_storage_metrics',
    'stg_usage_in_currency_daily',
    'stg_users',
    'stg_views',
    'stg_warehouse_events_history',
    'stg_warehouse_load_history',
    'target_role_grants',
    'warehouse_grants',
    'warehouse_params',
    'warehouse_usage_granted',
    'warehouses',
    'DAILY_RATES'
]

# to update from staging, dbt seed && dbt run --exclude staging daily_rates

time: 1.89 ms (started: 2023-06-06 17:45:19 -04:00)


In [4]:
run_sql("alter warehouse demo set warehouse_size=medium")

05:45:21 INFO:query: [alter warehouse demo set warehouse_size=medium]
05:45:22 INFO:query execution done


time: 398 ms (started: 2023-06-06 17:45:21 -04:00)


In [13]:
# df = sql_to_df("show tables in schema knot_share.optiml")
# df.name.to_list()

07:37:14 INFO:query: [show tables in schema knot_share.optiml]
07:37:14 INFO:query execution done


['ACTIVE_TIME_ESTIMATE',
 'BASE_OBJECT_ACCESS_EVENT',
 'BILLING_ESTIMATE',
 'CLIENT_TYPE_MAPPING',
 'CLIENT_VERSION_DEPRECATION_DATE',
 'CLONE_GROUP_STORAGE_METRICS',
 'COST_PER_QUERY',
 'DAILY_RATES',
 'DIRECT_OBJECT_ACCESS_EVENT',
 'HELLO',
 'OBJECT_MODIFIED_BY_QUERY',
 'OBJECT_MODIFIED_EVENT',
 'QUERY_ERA',
 'QUERY_ERA_HOUR',
 'QUERY_ERA_TEST',
 'QUERY_HISTORY_ENRICHED',
 'SETUP_SUCCESS',
 'STG_ACCESS_HISTORY',
 'STG_GRANTS_TO_ROLES',
 'STG_QUERY_HISTORY',
 'STG_RATE_SHEET_DAILY',
 'STG_REMAINING_BALANCE_DAILY',
 'STG_SESSIONS',
 'STG_STORAGE_USAGE',
 'STG_TABLE_STORAGE_METRICS',
 'STG_USAGE_IN_CURRENCY_DAILY',
 'STG_USERS',
 'STG_VIEWS',
 'STG_WAREHOUSE_EVENTS_HISTORY',
 'STG_WAREHOUSE_LOAD_HISTORY',
 'STG_WAREHOUSE_METERING_HISTORY',
 'STORAGE_USAGE_BY_USAGE_DATE',
 'SUSPEND_LAG_BY_WH',
 'SUSPEND_LAG_BY_WH_SIM',
 'TABLE_NAME_ACCESS_METRICS',
 'TABLE_NAME_AGG',
 'TABLE_STORAGE_METRICS_ENRICHED',
 'TABLE_SUCCESS',
 'TARGET_ROLE_GRANTS',
 'TEST_RESULTS_60D',
 'WAREHOUSES',
 'WAREHOUS

time: 291 ms (started: 2023-06-03 19:37:14 -04:00)


In [ ]:
target_schema = "share_copy.knot"
for t in df.name.to_list():
    sql = f"create or replace table {target_schema}.{t} as select * from knot_share.optiml.{t}"
    run_sql(sql)

07:37:54 INFO:query: [create or replace table share_copy.knot.ACTIVE_TIME_ESTIMATE as select * from kn...]
07:37:56 INFO:query execution done
07:37:56 INFO:query: [create or replace table share_copy.knot.BASE_OBJECT_ACCESS_EVENT as select * fro...]
07:38:15 INFO:query execution done
07:38:15 INFO:query: [create or replace table share_copy.knot.BILLING_ESTIMATE as select * from knot_s...]
07:38:17 INFO:query execution done
07:38:17 INFO:query: [create or replace table share_copy.knot.CLIENT_TYPE_MAPPING as select * from kno...]
07:38:18 INFO:query execution done
07:38:18 INFO:query: [create or replace table share_copy.knot.CLIENT_VERSION_DEPRECATION_DATE as selec...]
07:38:19 INFO:query execution done
07:38:19 INFO:query: [create or replace table share_copy.knot.CLONE_GROUP_STORAGE_METRICS as select * ...]
07:38:20 INFO:query execution done
07:38:20 INFO:query: [create or replace table share_copy.knot.COST_PER_QUERY as select * from knot_sha...]
07:38:36 INFO:query execution done
07:38:

time: 4min 11s (started: 2023-06-03 19:37:54 -04:00)



KeyboardInterrupt



In [9]:
import concurrent.futures

target_schema = "share_copy.knot"

def copy_table(table):
    sql = f"create or replace table share_copy.knot.{table} as select * from knot_share.optiml.{table}"
    run_sql(sql)
    sql = f"create or replace table share_copy.knot_monitoring.{table} clone share_copy.knot.{table}"    
    run_sql(sql)

# List of arguments
arguments = staging_tables # df.name.to_list()

# Create a ThreadPoolExecutor with the desired number of workers
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Submit the tasks to the executor
    futures = [executor.submit(copy_table, arg) for arg in arguments]

    # Wait for the tasks to complete
    # concurrent.futures.wait(futures)
    # Process the completed futures
    for future in concurrent.futures.as_completed(futures):
        try:
            # Retrieve the result from the completed future
            result = future.result()
        except Exception as e:
            # Handle the exception or failure
            print(f"Exception occurred: {e}")

05:51:23 INFO:query: [create or replace table share_copy.knot.stg_rate_sheet_daily as select * from kn...]
05:51:23 INFO:query: [create or replace table share_copy.knot.stg_warehouse_metering_history as select...]
05:51:23 INFO:query: [create or replace table share_copy.knot.stg_access_history as select * from knot...]
05:51:23 INFO:query: [create or replace table share_copy.knot.stg_grants_to_roles as select * from kno...]
05:51:25 INFO:query execution done
05:51:25 INFO:query: [create or replace table share_copy.knot_monitoring.stg_rate_sheet_daily clone sh...]
05:51:25 INFO:query execution done
05:51:25 INFO:query: [create or replace table share_copy.knot_monitoring.stg_warehouse_metering_histor...]
05:51:25 INFO:query execution done
05:51:25 INFO:query: [create or replace table share_copy.knot_monitoring.stg_grants_to_roles clone sha...]
05:51:25 INFO:query execution done
05:51:25 INFO:query: [create or replace table share_copy.knot.stg_query_history as select * from knot_...]
05:5

time: 3min 51s (started: 2023-06-06 17:51:23 -04:00)


In [8]:
run_sql("create or replace schema share_copy.knot_monitoring clone share_copy.knot;")

10:45:08 INFO:query: [create or replace schema share_copy.knot_monitoring clone share_copy.knot;]
10:45:23 INFO:query execution done


time: 15.2 s (started: 2023-06-03 22:45:08 -04:00)


In [7]:
import re

sql_statement = "insert into blah.blah.table as select * from a.b.c where column = 'value'"
modified_sql = re.sub(r'(\b(?:FROM|JOIN|INTO|TABLE)\s+)\w+\.\w+\.', r'\1', sql_statement, flags=re.IGNORECASE)

print("Modified SQL statement:", modified_sql)

Modified SQL statement: insert into table as select * from c where column = 'value'


In [8]:
import re

sql_statement = "select * from a.b.c where column = 'value'"
modified_sql = re.sub(r'\b(?:FROM|JOIN|INTO|TABLE)\s+\w+(?:\.\w+)+\.', '', sql_statement, flags=re.IGNORECASE)

print("Modified SQL statement:", modified_sql)

Modified SQL statement: select * c where column = 'value'


In [18]:
import re

text = "insert into hi.you.t select * from a.b.c join x.y.z"
pattern = r'\b(\w+\.\w+\.)'

matches = re.findall(pattern, text)

print(matches)

modified_sql = re.sub(pattern, '', text, flags=re.IGNORECASE)
print(modified_sql)

['hi.you.', 'a.b.', 'x.y.']
insert into t select * from c join z


NameError: name 'parsed' is not defined